In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from catboost import CatBoostClassifier
import optuna
from sklearn.neural_network import MLPClassifier

In [2]:
dataset_df = pd.read_csv('train.csv')
dataset_df['EJ'] = dataset_df['EJ'].replace({'A':0, 'B':1})
dataset_df = dataset_df.iloc[:,1:]

In [3]:
feature_columns = dataset_df.columns[:-1]  # 假设最后一列为目标变量，不需要处理

# 针对每个特征列，将无穷大值替换为前 97% 大的值
for column in feature_columns:
    # 计算前 97% 大的值
    percentile = 0.98
    threshold = dataset_df[column].quantile(percentile)
    
    # 将无穷大值替换为前 97% 大的值
    dataset_df[column] = dataset_df[column].replace(np.inf, threshold)

In [4]:
# 计算特征的均值
mean_values = dataset_df.mean()
# 填充缺失值为特征的均值
dataset_df = dataset_df.fillna(mean_values)

In [5]:
# 分割特征和类别
X = dataset_df.iloc[:, :-1]  # 特征
y = dataset_df.iloc[:, -1]   # 类别
X = X.dropna()  # 删除包含缺失值的行
y = y[X.index]  # 保持与特征对应的类别

# 对非数值类型特征进行独热编码
X_encoded = pd.get_dummies(X)

# 初始化特征选择器
k = 50  # 选择前k个重要特征
selector = SelectKBest(score_func=mutual_info_classif, k=k)

# 特征选择
X_selected = selector.fit_transform(X_encoded, y)

# 获取选择的特征索引
selected_feature_indices = selector.get_support(indices=True)

# 获取选择的特征名称
selected_features = X_encoded.columns[selected_feature_indices]
# selected_features = selected_features[:-2].append(pd.Index(['EJ']))
# 输出选择的特征
print(selected_features)

Index(['AB', 'AF', 'AH', 'AM', 'AX', 'AY', 'BC', 'BD ', 'BN', 'BP', 'BQ', 'BR',
       'CC', 'CD ', 'CF', 'CL', 'CR', 'CS', 'CW ', 'DA', 'DE', 'DF', 'DH',
       'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB', 'EE', 'EG', 'EH', 'EJ', 'EL',
       'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL', 'FR', 'FS', 'GB', 'GE', 'GF',
       'GH', 'GI', 'GL'],
      dtype='object')


In [6]:
X_train, y_train = dataset_df.loc[:, selected_features], dataset_df.iloc[:, -1]

In [7]:
def objective(trial):
    param = {
        'objective':'binary',  # 二分类目标函数
         'metric':'binary_logloss',  # 二分类损失函数
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 42
    }
    
    model = lgb.LGBMClassifier(**param)
    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=20, verbose=False)
    
    print(y_val)
    
    y_pred = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(classification_report(y_val, y_pred))
    
    return accuracy

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best trial:', study.best_trial.params)
print('Best accuracy:', study.best_value)

[I 2023-07-14 10:01:19,636] A new study created in memory with name: no-name-1d5439f6-b4ea-4da3-b59b-8457ef85338a
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/ta

C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
C:

[LightGBM] [Warning] feature_fraction is set=0.2888673246198624, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2888673246198624
[LightGBM] [Warning] bagging_fraction is set=0.9254751331551407, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9254751331551407
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
49     1
581    0
82     0
304    1
109    1
      ..
6      0
104    0
114    0
158    0
181    1
Name: Class, Length: 124, dtype: int64
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       101
           1       0.85      0.74      0.79        23

    accuracy                           0.93       124
   macro avg       0.90      0.85      0.87       124
weighted avg       0.93      0.93      0.93       124

[LightGBM] [Warning] feature_fraction is set=0.22397390689287294, colsample_bytree=1.0 will be ignored. Current value: feature

[I 2023-07-14 10:01:19,827] Trial 4 finished with value: 0.8306451612903226 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 153, 'learning_rate': 0.011040967385306969, 'feature_fraction': 0.6932335026218471, 'bagging_fraction': 0.884681360625414, 'bagging_freq': 7, 'min_child_samples': 80}. Best is trial 0 with value: 0.9274193548387096.
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_frac

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-14 10:01:19,944] Trial 8 finished with value

...

[LightGBM] [Warning] feature_fraction is set=0.5405435787565368, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5405435787565368
[LightGBM] [Warning] bagging_fraction is set=0.8383085827701982, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8383085827701982
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
49     1
581    0
82     0
304    1
109    1
      ..
6      0
104    0
114    0
158    0
181    1
Name: Class, Length: 124, dtype: int64
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       101
           1       1.00      0.65      0.79        23

    accuracy                           0.94       124
   macro avg       0.96      0.83      0.88       124
weighted avg       0.94      0.94      0.93       124

[LightGBM] [Warning] feature_fraction is set=0.6542849394459654, colsample_bytree=1.0 will be ignored. Current value: feature_

C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is de

49     1
581    0
82     0
304    1
109    1
      ..
6      0
104    0
114    0
158    0
181    1
Name: Class, Length: 124, dtype: int64
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       101
           1       1.00      0.74      0.85        23

    accuracy                           0.95       124
   macro avg       0.97      0.87      0.91       124
weighted avg       0.95      0.95      0.95       124

[LightGBM] [Warning] feature_fraction is set=0.6209641322762216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6209641322762216
[LightGBM] [Warning] bagging_fraction is set=0.9762253638070799, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9762253638070799
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
49     1
581    0
82     0
304    1
109    1
      ..
6      0
104    0
114    0
158    0
181    1
Name: Class, Length: 124, d

C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\3982982003.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
C:

49     1
581    0
82     0
304    1
109    1
      ..
6      0
104    0
114    0
158    0
181    1
Name: Class, Length: 124, dtype: int64
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       101
           1       0.94      0.70      0.80        23

    accuracy                           0.94       124
   macro avg       0.94      0.84      0.88       124
weighted avg       0.94      0.94      0.93       124

Best trial: {'boosting_type': 'dart', 'num_leaves': 72, 'learning_rate': 0.09373969766109666, 'feature_fraction': 0.6534737642936284, 'bagging_fraction': 0.9335986415575436, 'bagging_freq': 1, 'min_child_samples': 7}
Best accuracy: 0.967741935483871


In [33]:
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'random_state': 42,
        'verbose': False
    }
    
    model = CatBoostClassifier(**param)
    
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20)
    
    y_pred = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best trial:', study.best_trial.params)
print('Best accuracy:', study.best_value)

[I 2023-07-06 17:24:16,359] A new study created in memory with name: no-name-8cdd0a8d-3233-4e2b-99d8-a987a1f6e461
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:16,955] Trial 0 finished with value: 0.9696969696969697 and parameters: {'iterations': 402, 'learning_rate': 0.06392516479874999, 'depth': 5, 'l2_leaf_reg': 0.7000712708768574, 'borde

[I 2023-07-06 17:24:20,758] Trial 8 finished with value: 0.8585858585858586 and parameters: {'iterations': 748, 'learning_rate': 0.1940080777235773, 'depth': 10, 'l2_leaf_reg': 0.0053042241197440426, 'border_count': 207}. Best is trial 0 with value: 0.9696969696969697.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:21,622] Trial 9 finished wi

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:24,402] Trial 18 finished with value: 0.9696969696969697 and parameters: {'iterations': 528, 'learning_rate': 0.05169125847806759, 'depth': 6, 'l2_leaf_reg': 0.4112017663673828, 'border_count': 72}. Best is trial 14 with value: 0.9797979797979798.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1

[I 2023-07-06 17:24:28,298] Trial 26 finished with value: 0.9595959595959596 and parameters: {'iterations': 634, 'learning_rate': 0.07087429510982574, 'depth': 5, 'l2_leaf_reg': 0.08933143408060616, 'border_count': 134}. Best is trial 14 with value: 0.9797979797979798.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:28,568] Trial 27 finished w

[I 2023-07-06 17:24:30,977] Trial 35 finished with value: 0.9696969696969697 and parameters: {'iterations': 748, 'learning_rate': 0.08989893845909949, 'depth': 6, 'l2_leaf_reg': 0.5184249689163587, 'border_count': 144}. Best is trial 14 with value: 0.9797979797979798.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:31,434] Trial 36 finished wi

...

[I 2023-07-06 17:24:42,988] Trial 71 finished with value: 0.9595959595959596 and parameters: {'iterations': 689, 'learning_rate': 0.10251633636394965, 'depth': 5, 'l2_leaf_reg': 0.5149065522828117, 'border_count': 68}. Best is trial 63 with value: 0.98989898989899.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:43,318] Trial 72 finished with 

[I 2023-07-06 17:24:47,657] Trial 80 finished with value: 0.9494949494949495 and parameters: {'iterations': 374, 'learning_rate': 0.20703786956336379, 'depth': 4, 'l2_leaf_reg': 0.3866502366240121, 'border_count': 33}. Best is trial 63 with value: 0.98989898989899.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:48,085] Trial 81 finished with 

[I 2023-07-06 17:24:51,240] Trial 89 finished with value: 0.9494949494949495 and parameters: {'iterations': 608, 'learning_rate': 0.12881728653816843, 'depth': 8, 'l2_leaf_reg': 0.18058107267456466, 'border_count': 177}. Best is trial 63 with value: 0.98989898989899.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:51,581] Trial 90 finished wit

[I 2023-07-06 17:24:56,428] Trial 98 finished with value: 0.9696969696969697 and parameters: {'iterations': 518, 'learning_rate': 0.07323498523685866, 'depth': 10, 'l2_leaf_reg': 0.12194624678069056, 'border_count': 102}. Best is trial 63 with value: 0.98989898989899.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\1870400221.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2023-07-06 17:24:56,727] Trial 99 finished wi

Best trial: {'iterations': 626, 'learning_rate': 0.09949903256039326, 'depth': 4, 'l2_leaf_reg': 0.2707044422642283, 'border_count': 66}
Best accuracy: 0.98989898989899


In [40]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=12)
# 定义目标函数
def objective(trial):
    # 定义超参数搜索空间
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'random_state': 42
    }

    # 创建 XGBoost 模型
    model = xgb.XGBClassifier(**params)

    # 拟合模型
    model.fit(X_train, y_train)

    # 在验证集上进行预测
    y_pred = model.predict(X_val)

    # 计算准确率
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

# 创建 Optuna study 并运行优化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 输出最佳超参数和准确率
print('Best trial:', study.best_trial.params)
print('Best accuracy:', study.best_value)

[I 2023-07-06 17:49:04,991] A new study created in memory with name: no-name-6e931b45-e488-40f0-8300-9b5b83b54586
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float inste

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

[I 2023-07-06 17:49:05,550] Trial 11 finished with value: 0.8571428571428571 and parameters: {'eta': 0.09513327484532379, 'max_depth': 4, 'subsample': 0.7066101645932694, 'colsample_bytree': 0.7882425509870188, 'gamma': 2.388926239996815e-05}. Best is trial 0 with value: 0.9365079365079365.
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: Futur

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
[I 2023-07-06 17:49:06,251] Tr

...

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.5),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipykernel_76868\2020704515.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
C:\Users\GP1\AppData\Local\Temp\ipy

Best trial: {'eta': 0.35057733295058185, 'max_depth': 5, 'subsample': 0.8661015108322478, 'colsample_bytree': 0.9316339561187831, 'gamma': 0.7738911600640906}
Best accuracy: 0.9365079365079365


In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=12)
def objective(trial):
    # 定义超参数搜索空间
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt']),
        'random_state': 42
    }

    # 创建随机森林模型
    model = RandomForestClassifier(**params)

    # 拟合模型
    model.fit(X_train, y_train)

    # 在验证集上进行预测
    y_pred = model.predict(X_val)

    # 计算准确率
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

# 创建 Optuna study 并运行优化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 输出最佳超参数和准确率
print('Best trial:', study.best_trial.params)
print('Best accuracy:', study.best_value)

[I 2023-07-06 17:47:47,144] A new study created in memory with name: no-name-8aa564f0-5254-4632-91d1-e418d6b71300
[I 2023-07-06 17:47:47,738] Trial 0 finished with value: 0.8227848101265823 and parameters: {'n_estimators': 700, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': 'auto'}. Best is trial 0 with value: 0.8227848101265823.
[I 2023-07-06 17:47:48,415] Trial 1 finished with value: 0.8227848101265823 and parameters: {'n_estimators': 800, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8227848101265823.
[I 2023-07-06 17:47:48,591] Trial 2 finished with value: 0.8227848101265823 and parameters: {'n_estimators': 200, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8227848101265823.
[I 2023-07-06 17:47:49,073] Trial 3 finished with value: 0.7974683544303798 and parameters: {'n_estimators': 600, 'max_depth': 3, 'm

[I 2023-07-06 17:48:12,223] Trial 33 finished with value: 0.8860759493670886 and parameters: {'n_estimators': 900, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:13,147] Trial 34 finished with value: 0.8607594936708861 and parameters: {'n_estimators': 1000, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:14,026] Trial 35 finished with value: 0.8607594936708861 and parameters: {'n_estimators': 900, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:14,810] Trial 36 finished with value: 0.8734177215189873 and parameters: {'n_estimators': 800, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto'}. Best is trial 22 with value: 0.8987341

[I 2023-07-06 17:48:37,403] Trial 66 finished with value: 0.8987341772151899 and parameters: {'n_estimators': 900, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'auto'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:38,084] Trial 67 finished with value: 0.8860759493670886 and parameters: {'n_estimators': 700, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:38,919] Trial 68 finished with value: 0.8607594936708861 and parameters: {'n_estimators': 900, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'auto'}. Best is trial 22 with value: 0.8987341772151899.
[I 2023-07-06 17:48:39,782] Trial 69 finished with value: 0.8860759493670886 and parameters: {'n_estimators': 900, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'auto'}. Best is trial 22 with value: 0.898734

[I 2023-07-06 17:49:04,967] Trial 99 finished with value: 0.8607594936708861 and parameters: {'n_estimators': 400, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 22 with value: 0.8987341772151899.


Best trial: {'n_estimators': 900, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Best accuracy: 0.8987341772151899


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=12)
# 定义目标函数
def objective(trial):
    # 定义超参数搜索空间
    param = {
        'hidden_layer_sizes': trial.suggest_categorical('hidden_layer_sizes', [(100,), (100, 30, 10)]),
        'activation': trial.suggest_categorical('activation', ['relu']),
        'solver': trial.suggest_categorical('solver', ['adam']),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
        'batch_size': trial.suggest_categorical('batch_size', ['auto']),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant']),
        'learning_rate_init': trial.suggest_loguniform('learning_rate_init', 1e-5, 1e-1),
        'power_t': trial.suggest_uniform('power_t', 0.1, 0.9),
        'max_iter': trial.suggest_int('max_iter', 100, 500),
        'shuffle': trial.suggest_categorical('shuffle', [True]),
        'random_state': trial.suggest_categorical('random_state', [None]),
        'tol': trial.suggest_loguniform('tol', 1e-5, 1e-3),
        'verbose': trial.suggest_categorical('verbose', [False]),
        'warm_start': trial.suggest_categorical('warm_start', [False]),
        'momentum': trial.suggest_uniform('momentum', 0.7, 0.99),
        'nesterovs_momentum': trial.suggest_categorical('nesterovs_momentum', [True]),
        'early_stopping': trial.suggest_categorical('early_stopping', [True]),
        'validation_fraction': trial.suggest_uniform('validation_fraction', 0.1, 0.3),
        'beta_1': trial.suggest_uniform('beta_1', 0.8, 0.95),
        'beta_2': trial.suggest_uniform('beta_2', 0.99, 0.9999),
        'epsilon': trial.suggest_loguniform('epsilon', 1e-10, 1e-7),
        'n_iter_no_change': trial.suggest_int('n_iter_no_change', 5, 20),
        'max_fun': trial.suggest_int('max_fun', 10000, 20000)
    }
    
    # 创建 MLPClassifier 模型
    model = MLPClassifier(**param)
    
    # 拟合模型
    model.fit(X_train, y_train)
    
    # 预测验证集
    y_pred = model.predict(X_val)
    
    # 计算准确率作为目标函数的指标
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

# 创建 Optuna study 并运行优化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 输出最佳超参数和准确率
print('Best trial:', study.best_trial.params)
print('Best accuracy:', study.best_value)

[I 2023-07-14 10:14:29,468] A new study created in memory with name: no-name-0088780e-a75d-4c93-a8ee-ad276f5f53b9
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate_init': trial.suggest_loguniform('learning_rate_init', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optun

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
[I 2023-07-14 10:14:29,669] Trial 2 finished with value: 0.8354430379746836 and parameters: {'hidden_layer_sizes': (100, 30, 10), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.06532503943526648, 'batch_size': 'auto', 'learning_rate': 'constant', 'learning_rate_init': 2.5257461306546475e-05, 'power_t': 0.8464891783039725, 'max_iter': 232, 'shuffle': True, 'random_state': None, 'tol': 1.4267407822064375e-05, 'verbose': False, 'warm_start': False, 'momentum

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid featur

...

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 30, 10) which is of type tuple.
  warnings.warn(message)
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\GP1\AppData\Local\Temp\ipykernel_32604\1195639908.py:12: FutureWarning: suggest_logunifo

Best trial: {'hidden_layer_sizes': (100,), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.004524225053160557, 'batch_size': 'auto', 'learning_rate': 'constant', 'learning_rate_init': 0.0032695885785495216, 'power_t': 0.41229208337868917, 'max_iter': 391, 'shuffle': True, 'random_state': None, 'tol': 0.0002505082147304683, 'verbose': False, 'warm_start': False, 'momentum': 0.8862529058691623, 'nesterovs_momentum': True, 'early_stopping': True, 'validation_fraction': 0.1250369526043429, 'beta_1': 0.8337047263985674, 'beta_2': 0.9945669606589083, 'epsilon': 1.479694727954582e-09, 'n_iter_no_change': 19, 'max_fun': 16800}
Best accuracy: 0.8734177215189873


In [23]:
test_df = pd.read_csv('test.csv')
test_df = test_df.loc[:, selected_features]
test_df['EJ'] = test_df['EJ'].replace({'A':0, 'B':1})

In [24]:
y_pred = []
for model in Models:
    y_pred_proba = model.predict_proba(test_df)[:, 1]  # 取第一列的概率值（正类的概率）
    y_pred.append(y_pred_proba)

# 模型融合（平均概率值）
y_pred = np.mean(y_pred, axis=0)
print(y_pred)

[0.39831778 0.39831778 0.39831778 0.39831778 0.39831778]


In [25]:
def map_values(value,thred=best_threshold):
    """根据阈值映射"""
    if value <= thred:
        return 0.5 * (value / thred)
    else:
        return 0.5 + 0.5 * ((value - thred) / 0.65)
y_pred = np.vectorize(map_values)(y_pred)
pred = np.stack((1-y_pred, y_pred), axis=1)
print(pred)

[[0.42670194 0.57329806]
 [0.42670194 0.57329806]
 [0.42670194 0.57329806]
 [0.42670194 0.57329806]
 [0.42670194 0.57329806]]


In [26]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission[['class_0', 'class_1']] = pred
sample_submission.to_csv('submission.csv', index=False)

In [42]:
import numpy as np

def sigmoid_mapping(x):
    return 1 / (1 + np.exp(-10 * (x - 0.3)))

# 生成0到1的一组向量
x = np.linspace(0, 1, 100)

# 进行映射
mapped_x = sigmoid_mapping(x)

# 输出结果
print(x)
print(mapped_x)


[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898

In [45]:
import numpy as np

def custom_mapping(x):
    return np.where(x <= 0.3, 0.5 * (x / 0.3), 0.5 + 0.5 * ((x - 0.3) / 0.7))

# 生成0到1的一组向量
x = np.linspace(0, 1, 100)

# 进行映射
mapped_x = custom_mapping(x)

# 输出结果
print(x)
print(mapped_x)


[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898